# Pruning MobileNetV2 Model
This notebook shows how to reduce the size of a model by pruning its parameters. It assumes 
that a trained ```MobileNetV2``` model already exists in the ```Models``` directory. Please refer to the notebook
[Image Classification with MobileNetV2](MobileNetV2.ipynb) for more info about using a pretrained MobileNetV2 model.

If you want to prune a Low-Rank model, you can use [this](MobileNetV2-Reduce.ipynb) notebook
to reduce the number of parameters in ```MobileNetV2```.

## Load and evaluate the original pretrained model

In [1]:
from fireball import Model
from fireball.datasets.imagenet import ImageNetDSet
gpus='0,1,2,3'

# Create the test dataset for evaluation.
testDs = ImageNetDSet.makeDatasets('Test', batchSize=256, preProcessing='Crop256Tf', numWorkers=8)

# orgFileName = "Models/MobileNetV2.fbm"    # Original model
# orgFileName = "Models/MobileNetV2R.fbm"   # Reduced
orgFileName = "Models/MobileNetV2RR.fbm"    # Reduced - Retrained

model = Model.makeFromFile(orgFileName, testDs=testDs, gpus=gpus)
model.initSession()
model.printLayersInfo()
results = model.evaluate(topK=5)


Reading from "Models/MobileNetV2RR.fbm" ... Done.
Creating the fireball model "MobileNetV2" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 224x224x3    224 224 3     None                      0          
S1_L1_CONV       224 224 3     KSP: 3 2 0x1x0x1         112 112 32    None                      864        
S1_L2_BN         112 112 32                             112 112 32    ReLU     x<6.0            128        
S2_L1_DWCN       112 112 32    KSP: 3 1 s               112 112 32    None                      288        
S2_L2_BN         112 112 32                             112 112 32    ReLU     x<6.0            128        
S2_L3_CONV       112 112 32    KSP: 1 1 s               112 112 16    None                      512        
S2_L4_BN         112 112 16     

## Pruning the model
Here we prune the model using the ``pruneModel`` class method of the model.

In [2]:
prunedFileName = orgFileName.replace('.fbm', 'P.fbm')  # Append 'P' to the filename for "Pruned"
pResults = Model.pruneModel(orgFileName, prunedFileName, mseUb=.00005)


Reading model parameters from "Models/MobileNetV2RR.fbm" ... Done.
Pruning 264 tensors (mseUb=0.000050)
    Tensor 1 of 264 Shape: 3x3x3x32 ......... Done. 428 Pruned < 0.030167, MSE=0.000049, Reduced: 46.3%)
    Tensor 2 of 264 Shape: 32 ............... Ignored. (1-D Tensor)
    Tensor 3 of 264 Shape: 32 ............... Ignored. (1-D Tensor)
    Tensor 4 of 264 Shape: 32 ............... Ignored. (1-D Tensor)
    Tensor 5 of 264 Shape: 32 ............... Ignored. (1-D Tensor)
    Tensor 6 of 264 Shape: 3x3x32x1 ......... Done. 9 Pruned < 0.078409, MSE=0.000046, Reduced: -0.3%)
    Tensor 7 of 264 Shape: 32 ............... Ignored. (1-D Tensor)
    Tensor 8 of 264 Shape: 32 ............... Ignored. (1-D Tensor)
    Tensor 9 of 264 Shape: 32 ............... Ignored. (1-D Tensor)
    Tensor 10 of 264 Shape: 32 .............. Ignored. (1-D Tensor)
    Tensor 11 of 264 Shape: 1x1x32x16 ....... Done. 247 Pruned < 0.028462, MSE=0.000049, Reduced: 45.0%)
    Tensor 12 of 264 Shape: 16 .......

    Tensor 109 of 264 Shape: 384 ............ Ignored. (1-D Tensor)
    Tensor 110 of 264 Shape: 384 ............ Ignored. (1-D Tensor)
    Tensor 111 of 264 Shape: 3x3x384x1 ...... Done. 227 Pruned < 0.049420, MSE=0.000049, Reduced: 3.4%)
    Tensor 112 of 264 Shape: 384 ............ Ignored. (1-D Tensor)
    Tensor 113 of 264 Shape: 384 ............ Ignored. (1-D Tensor)
    Tensor 114 of 264 Shape: 384 ............ Ignored. (1-D Tensor)
    Tensor 115 of 264 Shape: 384 ............ Ignored. (1-D Tensor)
    Tensor 116 of 264 Shape: 1x1x384x64 ..... Done. 9695 Pruned < 0.019906, MSE=0.000050, Reduced: 36.3%)
    Tensor 117 of 264 Shape: 64 ............. Ignored. (1-D Tensor)
    Tensor 118 of 264 Shape: 64 ............. Ignored. (1-D Tensor)
    Tensor 119 of 264 Shape: 64 ............. Ignored. (1-D Tensor)
    Tensor 120 of 264 Shape: 64 ............. Ignored. (1-D Tensor)
    Tensor 121 of 264 Shape: 1x1x64x384 ..... Done. 9287 Pruned < 0.020290, MSE=0.000050, Reduced: 34.7%)
    

## Evaluate the pruned model
Compare the new number of parameters with the original. Let's see the impact of this reduction to the performance of the model.

In [3]:
model = Model.makeFromFile(prunedFileName, testDs=testDs, gpus=gpus)   
model.printLayersInfo()
model.initSession()
results = model.evaluate()


Reading from "Models/MobileNetV2RRP.fbm" ... Done.
Creating the fireball model "MobileNetV2" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 224x224x3    224 224 3     None                      0          
S1_L1_CONV       224 224 3     KSP: 3 2 0x1x0x1         112 112 32    None                      436        
S1_L2_BN         112 112 32                             112 112 32    ReLU     x<6.0            128        
S2_L1_DWCN       112 112 32    KSP: 3 1 s               112 112 32    None                      279        
S2_L2_BN         112 112 32                             112 112 32    ReLU     x<6.0            128        
S2_L3_CONV       112 112 32    KSP: 1 1 s               112 112 16    None                      265        
S2_L4_BN         112 112 16    

## Re-training after pruning
Here we make a new model for re-training from the file created above. We then call the ```train``` method of the model to start the re-training.

After re-training, we run the ```evaluate``` function again to see how the re-training improved the performance
of the model.

The re-trained model is then saved to a file appending an 'R' letter (for Re-trained) to the end of the pruned model file name.

In [4]:
tuneDs = ImageNetDSet.makeDatasets('tune', batchSize=256, preProcessing='Crop256Tf', numWorkers=8)
print(tuneDs)

model = Model.makeFromFile(prunedFileName, trainDs=tuneDs, testDs=testDs,
                           numEpochs=5,
                           learningRate=(0.00005, 0.000005),
                           optimizer="Adam",
                           gpus=gpus)
model.printLayersInfo()
model.initSession()
model.train()
results = model.evaluate(topK=5)

retrainedFileName = prunedFileName.replace('.fbm', 'R.fbm')  # Append 'R' to the filename for "Retrained"
model.save(retrainedFileName)

ImageNetDSet Dataset Info:
    Dataset Name ................................... tune
    Dataset Location ............................... /data/ImageNet/
    Number of Classes .............................. 1000
    Number of Samples .............................. 64000
    Sample Shape ................................... (224, 224, 3)
    Preprocessing .................................. Crop256Tf
    Number of Workers .............................. 8


Reading from "Models/MobileNetV2RRP.fbm" ... Done.
Creating the fireball model "MobileNetV2" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 224x224x3    224 224 3     None                      0          
S1_L1_CONV       224 224 3     KSP: 3 2 0x1x0x1         112 112 32    None                      436        
S1_L2_

## Where do I go from here?

[Quantizing MobileNetV2 Model](MobileNetV2-Quantize.ipynb)

[Exporting MobileNetV2 Model to ONNX](MobileNetV2-ONNX.ipynb)

[Exporting MobileNetV2 Model to TensorFlow](MobileNetV2-TF.ipynb)

[Exporting MobileNetV2 Model to CoreML](MobileNetV2-CoreML.ipynb)

---

[Fireball Playgrounds](../Contents.ipynb)

[Image Classification with MobileNetV2](MobileNetV2.ipynb)

[Reducing number of parameters of MobileNetV2 Model](MobileNetV2-Reduce.ipynb)